<a href="https://colab.research.google.com/github/mounikakarasu/ticketing-system/blob/main/00_sst2_finetune_baseline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# SST-2 Fine-Tune Baseline (BERT-base, 1 epoch)
This notebook fine-tunes a pretrained BERT model on the SST-2 sentiment classification dataset using the HuggingFace Trainer API.



In [15]:
import os
os.environ["WANDB_DISABLED"] = "true"
os.environ["WANDB_MODE"] = "disabled"
os.environ["WANDB_SILENT"] = "true"

In [1]:
!pip install -q "transformers[torch]" datasets evaluate accelerate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 5.5 MB/s eta 0:00:00


These imports provide the model, dataset utilities, evaluation metrics, and the Trainer API. Setting a seed ensures reproducible results

In [2]:
from datasets import load_dataset
import evaluate
import numpy as np

from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,
    DataCollatorWithPadding,
    Trainer,
    TrainingArguments,
    set_seed,
)

set_seed(42)

SST-2 (Stanford Sentiment Treebank) consists of movie review sentences labeled as positive (1) or negative (0). It is part of the GLUE benchmark.


In [3]:
dataset = load_dataset("glue", "sst2")
dataset

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md: 0.00B [00:00, ?B/s]

sst2/train-00000-of-00001.parquet:   0%|          | 0.00/3.11M [00:00<?, ?B/s]

sst2/validation-00000-of-00001.parquet:   0%|          | 0.00/72.8k [00:00<?, ?B/s]

sst2/test-00000-of-00001.parquet:   0%|          | 0.00/148k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/67349 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/872 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1821 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['sentence', 'label', 'idx'],
        num_rows: 67349
    })
    validation: Dataset({
        features: ['sentence', 'label', 'idx'],
        num_rows: 872
    })
    test: Dataset({
        features: ['sentence', 'label', 'idx'],
        num_rows: 1821
    })
})

Text must be converted into model-readable token IDs. We use the BERT tokenizer with truncation only.

In [4]:
checkpoint = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

def tokenize_function(batch):
    return tokenizer(batch["sentence"], truncation=True)

tokenized = dataset.map(tokenize_function, batched=True)
tokenized

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

Map:   0%|          | 0/67349 [00:00<?, ? examples/s]

Map:   0%|          | 0/872 [00:00<?, ? examples/s]

Map:   0%|          | 0/1821 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['sentence', 'label', 'idx', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 67349
    })
    validation: Dataset({
        features: ['sentence', 'label', 'idx', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 872
    })
    test: Dataset({
        features: ['sentence', 'label', 'idx', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 1821
    })
})

Dynamic padding ensures each batch pads to the longest sequence in that batch, improving efficiency

In [5]:
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

BERT-base-uncased is a 12-layer Transformer encoder pretrained on English text with masked-language modeling. A classification head is added for binary sentiment prediction.


In [6]:
model = AutoModelForSequenceClassification.from_pretrained(
    checkpoint,
    num_labels=2
)

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Accuracy is the standard metric for SST-2 validation.

In [7]:
accuracy = evaluate.load("accuracy")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    preds = np.argmax(logits, axis=-1)
    return accuracy.compute(predictions=preds, references=labels)

We train for a single epoch using standard BERT fine-tuning hyperparameters

In [16]:
args = TrainingArguments(
    output_dir="sst2-baseline-checkpoints",
    eval_strategy="epoch",
    save_strategy="epoch",               # <-- match eval strategy
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=1,
    weight_decay=0.01,
    load_best_model_at_end=True,
    metric_for_best_model="accuracy",
    push_to_hub=False,
)

trainer = Trainer(
    model=model,
    args=args,
    train_dataset=tokenized["train"],
    eval_dataset=tokenized["validation"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).
/tmp/ipython-input-1008526586.py:15: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


W&B and HuggingFace Hub logging were disabled intentionally, for reproducible offline training.

In [17]:
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy
1,0.165200,0.251019,0.916284


TrainOutput(global_step=4210, training_loss=0.2150039962804799, metrics={'train_runtime': 504.0319, 'train_samples_per_second': 133.621, 'train_steps_per_second': 8.353, 'total_flos': 1216670228308500.0, 'train_loss': 0.2150039962804799, 'epoch': 1.0})

- 1 epoch fine-tuning  
- batch size = 16  
- learning rate = 2e-5  
- dynamic padding  
- Trainer API  
- evaluation + checkpointing at each epoch  

In [18]:
results = trainer.evaluate()
results

{'eval_loss': 0.25101909041404724,
 'eval_accuracy': 0.9162844036697247,
 'eval_runtime': 2.2681,
 'eval_samples_per_second': 384.463,
 'eval_steps_per_second': 24.249,
 'epoch': 1.0}

In [19]:
print(f"Validation accuracy: {results['eval_accuracy']:.4f}")

Validation accuracy: 0.9163


Validation accuracy after 1 epoch: 0.9163